<a href="https://colab.research.google.com/github/Suresh045/TNSDC/blob/main/3_DC_Grid_Copy1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
#importing the Libraies
import numpy as np
import pandas as pd

In [25]:
# Reading the Dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

In [26]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [27]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [28]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [29]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [30]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [31]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 0.20, random_state = 0)


In [32]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [33]:
from sklearn.svm import SVC

In [34]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [35]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10, 100],                 # Regularization strength
    'kernel': ['linear', 'poly', 'rbf'],    # Type of kernel
    'degree': [2, 3, 4],                    # Only relevant for 'poly' kernel
    'gamma': ['scale', 'auto', 0.1, 0.01],  # Kernel coefficient
    'class_weight': [None, 'balanced']      # Handle class imbalance
}


grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100],
                         'class_weight': [None, 'balanced'],
                         'degree': [2, 3, 4],
                         'gamma': ['scale', 'auto', 0.1, 0.01],
                         'kernel': ['linear', 'poly', 'rbf']},
             scoring='f1_weighted', verbose=3)

In [36]:

re=grid.cv_results_
grid_predictions = grid.predict(X_test)

In [37]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

In [38]:
# print classification report
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [39]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'C': 100, 'class_weight': 'balanced', 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf'}: 0.950648360030511


In [40]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[55  3]
 [ 1 21]]


In [41]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.98      0.95      0.96        58
           1       0.88      0.95      0.91        22

    accuracy                           0.95        80
   macro avg       0.93      0.95      0.94        80
weighted avg       0.95      0.95      0.95        80



In [42]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

np.float64(0.9780564263322884)

In [43]:
# Assuming 'grid' is your GridSearchCV object that has been fitted to the data
best_model = grid.best_estimator_

In [44]:
import pickle
filename="RF_G.sav"
pickle.dump(best_model,open(filename,'wb'))

In [45]:
table=pd.DataFrame.from_dict(re)

In [46]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_degree,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.014124,0.001172,0.005810,0.001040,0.1,None,2,scale,linear,"{'C': 0.1, 'class_weight': None, 'degree': 2, ...",0.794154,0.736364,0.755046,0.820367,0.902824,0.801751,0.058441,177
1,0.018015,0.002383,0.005196,0.000670,0.1,None,2,scale,poly,"{'C': 0.1, 'class_weight': None, 'degree': 2, ...",0.689723,0.744951,0.759058,0.764620,0.738479,0.739366,0.026540,248
2,0.021241,0.000826,0.006006,0.000322,0.1,None,2,scale,rbf,"{'C': 0.1, 'class_weight': None, 'degree': 2, ...",0.875759,0.808442,0.858747,0.907389,0.968750,0.883817,0.053187,67
3,0.014111,0.002625,0.005086,0.000357,0.1,None,2,auto,linear,"{'C': 0.1, 'class_weight': None, 'degree': 2, ...",0.794154,0.736364,0.755046,0.820367,0.902824,0.801751,0.058441,177
4,0.016829,0.000300,0.005159,0.000191,0.1,None,2,auto,poly,"{'C': 0.1, 'class_weight': None, 'degree': 2, ...",0.689723,0.744951,0.759058,0.764620,0.738479,0.739366,0.026540,248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,0.021884,0.000590,0.004765,0.000057,100.0,balanced,4,0.1,poly,"{'C': 100, 'class_weight': 'balanced', 'degree...",0.761294,0.779162,0.779162,0.795968,0.750432,0.773204,0.015809,222
284,0.022312,0.001989,0.004883,0.000178,100.0,balanced,4,0.1,rbf,"{'C': 100, 'class_weight': 'balanced', 'degree...",0.860542,0.891049,0.907389,0.907389,0.968750,0.907024,0.035288,1
285,0.093381,0.019100,0.004614,0.000089,100.0,balanced,4,0.01,linear,"{'C': 100, 'class_weight': 'balanced', 'degree...",0.798515,0.782821,0.783907,0.815315,0.892138,0.814539,0.040555,102
286,0.018470,0.004597,0.004619,0.000876,100.0,balanced,4,0.01,poly,"{'C': 100, 'class_weight': 'balanced', 'degree...",0.461468,0.480769,0.480769,0.480769,0.480769,0.476909,0.007720,260
